In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark=SparkSession.builder.appName("Walmart Exercise").getOrCreate()

In [3]:
df=spark.read.csv("walmart_stock.csv",inferSchema='True',header='True')
df.show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000000004|
|2012-01-10|             59.43|59.709998999999996|             5

In [4]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [5]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

In [6]:
missing=df.select([F.count(F.when(df[i].isNull(),1)).alias(i) for i in df.columns])
missing.show()

+----+----+----+---+-----+------+---------+
|Date|Open|High|Low|Close|Volume|Adj Close|
+----+----+----+---+-----+------+---------+
|   0|   0|   0|  0|    0|     0|        0|
+----+----+----+---+-----+------+---------+



In [7]:
len(df.columns)

7

In [8]:
df.count()

1258

In [9]:
df.filter(df['High']>90).show()

+----------+---------+-----------------+---------+---------+--------+-----------------+
|      Date|     Open|             High|      Low|    Close|  Volume|        Adj Close|
+----------+---------+-----------------+---------+---------+--------+-----------------+
|2015-01-08|89.209999|90.66999799999999|    89.07|90.470001|12713600|84.91421600000001|
|2015-01-09|    90.32|        90.389999|    89.25|89.349998| 8522500|        83.862993|
|2015-01-12|89.360001|        90.309998|89.220001|90.019997| 7372500|        84.491846|
|2015-01-13|90.800003|        90.970001|    88.93|89.309998| 8215400|        83.825448|
+----------+---------+-----------------+---------+---------+--------+-----------------+



In [10]:
df.filter((df['Date']>= '2012-01-10') & (df['Date']<='2012-01-20')).show()


+----------+------------------+------------------+------------------+------------------+--------+------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-10|             59.43|59.709998999999996|             58.98|59.040001000000004| 6907300|         51.494109|
|2012-01-11|         59.060001|         59.529999|59.040001000000004|         59.400002| 6365600|         51.808098|
|2012-01-12|59.790001000000004|              60.0|         59.400002|              59.5| 7236400|51.895315999999994|
|2012-01-13|             59.18|59.610001000000004|59.009997999999996|59.540001000000004| 7729300|51.930203999999996|
|2012-01-17|         59.869999|60.110001000000004|             59.52|         59.849998| 8500000|         52.200581|
|2012-01-18|59.790001000000004|         60.029999|         59.65

In [11]:
df.agg(F.avg('Close').alias("Average")).show()

+-----------------+
|          Average|
+-----------------+
|72.38844998012726|
+-----------------+



In [12]:
df_year=df.withColumn("Year", F.year("Date"))
df_year.show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+----+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|Year|
+----------+------------------+------------------+------------------+------------------+--------+------------------+----+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|2012|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|2012|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|2012|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|2012|
|2012-01-09|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000000004|2012|
|2012-01-10|            

In [13]:
df_year.groupBy('Year').agg(F.avg('Close').alias('Average Close Price')).show()

+----+-------------------+
|Year|Average Close Price|
+----+-------------------+
|2015|  72.49111121825403|
|2013|  75.32051611904767|
|2014|  77.32738088888888|
|2012|  67.21512003200003|
|2016|  69.54706346825398|
+----+-------------------+



In [14]:
df_filter=df.filter(df['Close']>60)
df_filter.show(2)
df_filter.sort('Date',ascending=False).show()


+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-18|59.790001000000004|60.029999|59.650002|60.009997999999996| 5911400|         52.340131|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 2 rows

+----------+-----------------+---------+-----------------+-----------------+--------+-----------------+
|      Date|             Open|     High|              Low|            Close|  Volume|        Adj Close|
+----------+-----------------+---------+-----------------+-----------------+--------+-----------------+
|2016-12-30|        69.120003|    69.43|68.83000200000001|        69.

In [15]:
maxi=df.agg(F.max('Close').alias("Maximum Close Price"))
maxi.show()
mini=df.agg(F.min('Close').alias("Minimum Close Price"))
mini.show()

+-------------------+
|Maximum Close Price|
+-------------------+
|          90.470001|
+-------------------+

+-------------------+
|Minimum Close Price|
+-------------------+
|          56.419998|
+-------------------+



In [16]:
df=df.withColumn("Price Difference", df['High']-df['Low']).show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+-------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|   Price Difference|
+----------+------------------+------------------+------------------+------------------+--------+------------------+-------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996| 1.1900019999999998|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475| 0.8799969999999959|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|               1.25|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922| 0.5800020000000004|
|2012-01-09|         59.029999|         5

In [17]:

df_year.show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+----+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|Year|
+----------+------------------+------------------+------------------+------------------+--------+------------------+----+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|2012|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|2012|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|2012|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|2012|
|2012-01-09|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000000004|2012|
|2012-01-10|            

In [18]:
count=df_year.groupBy('Year').agg(F.count('Open').alias('Number of Records'))
count.show()

+----+-----------------+
|Year|Number of Records|
+----+-----------------+
|2015|              252|
|2013|              252|
|2014|              252|
|2012|              250|
|2016|              252|
+----+-----------------+



In [19]:
df_month=df_year.withColumn("Month",F.month('Date'))
df_month.show(2)

+----------+------------------+---------+---------+------------------+--------+------------------+----+-----+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|Year|Month|
+----------+------------------+---------+---------+------------------+--------+------------------+----+-----+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|2012|    1|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|2012|    1|
+----------+------------------+---------+---------+------------------+--------+------------------+----+-----+
only showing top 2 rows



In [20]:
df_month.groupBy('Month').agg(F.avg('Volume').alias('Average Volume')).show()

+-----+-----------------+
|Month|   Average Volume|
+-----+-----------------+
|   12|7967959.433962264|
|    1|8761851.485148516|
|    6|8303756.603773585|
|    3|7721836.448598131|
|    5|8632350.943396226|
|    9|7194450.980392157|
|    4|7956230.476190476|
|    8|7745198.181818182|
|    7|6841084.112149533|
|   10|9008785.454545455|
|   11|8921626.732673267|
|    2|9764147.422680411|
+-----+-----------------+



In [21]:
rows=df_month.filter(df_month['Close']>60).count()
rows

1177

In [22]:
df_year.groupBy('Year').agg(F.max('Close').alias('Maximum')).show()

+----+---------+
|Year|  Maximum|
+----+---------+
|2015|90.470001|
|2013|81.209999|
|2014|87.540001|
|2012|77.150002|
|2016|74.300003|
+----+---------+



In [23]:
df_year.groupBy('Year').agg(F.min('Close').alias('Minimum')).show()

+----+------------------+
|Year|           Minimum|
+----+------------------+
|2015|         56.419998|
|2013|         68.300003|
|2014|         72.660004|
|2012|57.360001000000004|
|2016|             60.84|
+----+------------------+



In [24]:
df_month.orderBy(df_month['Volume'].desc()).show(5)

+----------+------------------+------------------+------------------+------------------+--------+---------+----+-----+
|      Date|              Open|              High|               Low|             Close|  Volume|Adj Close|Year|Month|
+----------+------------------+------------------+------------------+------------------+--------+---------+----+-----+
|2015-10-14|         66.610001|         67.949997|             60.02|         60.029999|80898100|57.429177|2015|   10|
|2015-10-15|         59.700001|         60.470001|58.610001000000004|         59.330002|46253500|56.759508|2015|   10|
|2012-04-23|         59.150002|60.240002000000004|         59.130001|59.540001000000004|38007300| 52.27779|2012|    4|
|2016-05-19|         68.099998| 69.33000200000001|              67.5|         69.199997|35076700| 67.74298|2016|    5|
|2012-04-24|59.259997999999996|         59.349998|         57.650002|             57.77|30090600| 50.72368|2012|    4|
+----------+------------------+-----------------